# Eliminación de preguntas repetidas
Ya con todas las preguntas que cumplen los requisitos del INE para ser tomadas en cuenta, vamos 
a hacer otra depuración para eliminar aquellas preguntas que están repetidas.
Es importante considerar que dos preguntas pueden ser iguales aún cuando estén formuladas con 
palabras diferentes (es decir, una puede ser paráfrasis de la otra).
El resultado de este paso serán dos archivos:

- Uno que contenga el texto de las preguntas repetidas, así como una métrica que mida su parecido. De estas preguntas, sólo se conservará 1 en el dataset de trabajo.

  - La pregunta que será conservada es aquella que fue enviada primero de acuerdo al timestamp
          
- Un archivo con las preguntas depuradas (eliminando las repeticiones)

In [12]:
!pip install spacy
!pip install scikit-learn
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     --------------------------------------- 0.0/42.3 MB 217.9 kB/s eta 0:03:14
     --------------------------------------- 0.1/42.3 MB 327.7 kB/s eta 0:02:09
     --------------------------------------- 0.1/42.3 MB 595.3 kB/s eta 0:01:11
     ---------------------------------------- 0.3/42.3 MB 1.4 MB/s eta 0:00:31
      --------------------------------------- 0.6/42.3 MB 2.4 MB/s eta 0:00:18
     - -------------------------------------- 1.4/42.3 MB 4.4 MB/s eta 0:00:10
     -- ------------------------------------- 2.7/42.3 MB 7.4 MB/s eta 0:00:06
     --- ------------------------------------ 4.2/42.3 MB 10.6 MB/s eta 0:00:04
     ----- ---------------------------------- 6.2/42.3 MB 14.1 MB/s eta 0:00:03
     ------- -------------------------------- 8.3/42.3 MB 16.5 MB/s eta 0:00:03
     --------- ----------------------------- 10.3/42.3 MB 26

1. Se importa los modulos necesarios para esta etapa

In [13]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import re
from utils import save_df

nlp = spacy.load('es_core_news_md')

2. Se importan las preguntas restantes del paso previo

In [4]:
limpio_df = pd.read_csv('./out_datasets/2-4_base_limpia.csv')

3. Se vulve a normalizas las preguntas restantes.

In [6]:
def replace_accented(match):
    accented = 'áéíóúñüÁÉÍÓÚÑÜ'
    unaccented = 'aeiounuAEIOUNU'
    trans_table = str.maketrans(accented, unaccented)
    return match.translate(trans_table)

stop_words = nltk.corpus.stopwords.words('spanish')
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\sáéíóúñüÁÉÍÓÚÑÜ]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [replace_accented(token) for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [7]:
limpio_df['pregunta_norm'] = list(map(normalize_document, limpio_df['pregunta']))
limpio_df['pregunta_norm'] = limpio_df['pregunta_norm'].astype(str)

In [8]:
display(limpio_df)

,Unnamed: 0,entidad,edad,genero,grupo_discriminacion,tema,pregunta,fecha,hate_speech,pregunta_norm
0,1,Aguascalientes,14,Masculino,No Aplica,Salud,En México muchas personas se quejan del sistem...,2024-03-22 00:04:00,NaN,mexico muchas personas quejan sistema salud co...
1,2,Nuevo León,17,Masculino,Personas de la diversidad sexual,Educación,¿Qué estrategias y planes concretos se propone...,2024-03-22 00:02:00,NaN,estrategias planes concretos proponen asegurar...
2,3,México,34,Femenino,No Aplica,Salud,Dentro de su plan de gobierno ¿qué acciones es...,2024-03-22 00:01:00,NaN,dentro plan gobierno acciones dirigidas crear ...
3,4,México,58,Masculino,Personas con discapacidad,Combate a la corrupción,"Por qué ,sigue la corrupción a pesar se, saber...",2024-03-21 23:59:00,NaN,sigue corrupcion pesar saber donde si origuina
4,5,Guanajuato,40,Masculino,No Aplica,Transparencia,Quisiera participar en estás elecciones,2024-03-21 23:58:00,NaN,quisiera participar elecciones
...,...,...,...,...,...,...,...,...,...,...
15422,22882,Ciudad de México,21,Masculino,No Aplica,Educación,¿En algunos sectores de México hay más univers...,2024-02-21 09:19:00,NaN,sectores mexico universidades privadas publica...
15423,22883,Ciudad de México,57,Masculino,No Aplica,Violencia en contra de las mujeres,Que dependencia u organismo se hará responsabl...,2024-02-20 20:11:00,NaN,dependencia u organismo hara responsable acomp...
15424,22884,Baja California,68,Masculino,No Aplica,Salud,Qué porcentaje del Presupuesto de Egresos de l...,2024-02-20 20:06:00,NaN,porcentaje presupuesto egresos federacion dest...
15425,22886,Sonora,44,Femenino,Personas afromexicanas,No discriminación y grupos vulnerables,Que hará para poder favorecer a los mexicanos ...,2024-02-20 12:27:00,NaN,hara poder favorecer mexicanos vulnerables alg...


4. Se procesan las pregunas ya normalizadas y se calcula la similitud entre las preguntas

In [9]:
preguntas = limpio_df['pregunta_norm']

vectores = [nlp(str(pregunta)).vector for pregunta in preguntas]

similitud = cosine_similarity(vectores)

5. Ya con la similitud calculada se comparan todas contra todas y se elimina la pregunta mas reciente y parecida.

In [10]:
repetidas = []

for index, fila in limpio_df.iterrows():
    for j in range(index + 1, len(preguntas)):
        if similitud[index, j] > 0.9:
            p1_fecha = limpio_df.loc[index]['fecha']
            p2_fecha = limpio_df.loc[j]['fecha']
            if p1_fecha < p2_fecha:
              repetidas.append(pd.DataFrame([[limpio_df.iloc[j]['pregunta'], limpio_df.loc[index]['pregunta']]], index=None))
            else:
              repetidas.append(pd.DataFrame([[limpio_df.loc[index]['pregunta'], limpio_df.iloc[j]['pregunta']]], index=None))
repetidas = pd.concat(repetidas)
repetidas.columns = ['repetida_de', 'pregunta']

In [11]:
display(repetidas)

,repetida_de,pregunta
0,En México muchas personas se quejan del sistem...,Cómo podría usted cambiar el sistema de salud ...
0,En México muchas personas se quejan del sistem...,¿Cómo podrías incrementar la eficacia de los ...
0,En México muchas personas se quejan del sistem...,Que nos digan su opinión con respecto al servi...
0,En México muchas personas se quejan del sistem...,El sistema de salud es insuficiente mi pregunt...
0,En México muchas personas se quejan del sistem...,¿Cuál es su plan concreto para mejorar el acce...
...,...,...
0,¿Cuál sería su enfoque para abordar la violenc...,¿Cuál es su plan para abordar de manera efecti...
0,¿Cómo abordará el problema de la desigualdad e...,¿Cómo abordará la brecha educativa entre zonas...
0,¿Cuál es su enfoque para abordar la violencia ...,¿Cuál es la estrategia proponen para prevenir ...
0,"¿Cuáles son sus propuestas para concientizar, ...",¿Qué estrategias implementarán para sensibiliz...


6. Por ultimo se guarda el DF

In [16]:
save_df(repetidas, "3_preguntas_repetidas.csv", index=False)

In [18]:
len(set(repetidas['repetida_de']))

6161